In [6]:
import pandas as pd
import requests
from io import BytesIO
from datetime import datetime
from pandas.tseries.offsets import CustomBusinessDay
import holidays
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
us_bd = CustomBusinessDay(calendar=holidays.US())
look_back = 30

In [62]:
def hist_trace(days=look_back):
    
    data_frames = []
    
    business_dates = pd.date_range(end=datetime.now(), periods=days+1, freq=us_bd)
    
    for date in business_dates:
        date_str = date.strftime('%Y-%m-%d')
        url = f"https://cdn.finra.org/trace/treasury-aggregates/daily/ts-daily-aggregates-{date_str}.xlsx"
        
        try:
            response = requests.get(url)
            response.raise_for_status()  
            df = pd.read_excel(BytesIO(response.content), header=[3, 4])

            # Clean the DataFrame
            df = df.drop(columns=[('Total', 'VWAP')])
            tips_index = df[df[('Category', 'Unnamed: 0_level_1')] == 'TIPS'].index[0]
            df = df.loc[tips_index:].dropna(axis=0)
            df = df[df[('Category', 'Unnamed: 0_level_1')] != 'Total'].reset_index(drop=True)
            df['Date'] = date_str
            
            data_frames.append(df)
        
        except requests.exceptions.HTTPError:
            print(f"Data for {date_str} is not available. Skipping this date.")
            continue
    
    if not data_frames:
        print("No data available for the specified date range.")
        return None
    
    combined_df = pd.concat(data_frames).reset_index(drop=True)

    # for easier display
    combined_df[('Category', 'Unnamed: 0_level_1')] = combined_df[('Category', 'Unnamed: 0_level_1')].replace('> 5 years and <= 10 years', '> 5 and <= 10 years')

    return combined_df

def get_subcat_rows(df, category_value, num_following=2):
    index_positions = df.index[df[('Category', 'Unnamed: 0_level_1')] == category_value]
    return pd.concat([df.iloc[i:i + num_following + 1] for i in index_positions]).reset_index(drop=True)


In [74]:
def plot_trades_and_par(df, title, look_back=30):
    last_date = df['Date'].max()
    categories = df[('Category', 'Unnamed: 0_level_1')].unique()
    
    # Define segments to be plotted as columns
    segments = ['Total', 'ATS & Interdealer', 'Dealer to Customer']
    
    # Initialize dictionaries to store the data for each segment
    trades_last_close = {segment: [] for segment in segments}
    trades_avg = {segment: [] for segment in segments}
    par_last_close = {segment: [] for segment in segments}
    par_avg = {segment: [] for segment in segments}

    for category in categories:
        category_data = df[df[('Category', 'Unnamed: 0_level_1')] == category]
        
        if not category_data.empty:
            for segment in segments:
                # Trades values
                trades_last_close_value = category_data[category_data['Date'] == last_date][(segment, 'Trades')].values[0]
                trades_avg_value = category_data[category_data['Date'] < last_date][(segment, 'Trades')].mean()
                
                trades_last_close[segment].append(trades_last_close_value)
                trades_avg[segment].append(trades_avg_value)

                # Par Value values
                par_last_close_value = category_data[category_data['Date'] == last_date][(segment, 'Par Value')].values[0]
                par_avg_value = category_data[category_data['Date'] < last_date][(segment, 'Par Value')].mean()
                
                par_last_close[segment].append(par_last_close_value)
                par_avg[segment].append(par_avg_value)
    
    # Create subplots: 2 rows (Trades, Par Value) and 3 columns (Total, ATS & Interdealer, Dealer to Customer)
    fig = make_subplots(rows=2, cols=3, subplot_titles=[f'{segment}' for segment in segments] * 2)
    
    # Add bars for each segment in the "Trades" row (row 1)
    for i, segment in enumerate(segments):
        fig.add_trace(go.Bar(name='Last Close', x=categories, y=trades_last_close[segment]), row=1, col=i+1)
        fig.add_trace(go.Bar(name=f'{look_back}-Day Avg', x=categories, y=trades_avg[segment]), row=1, col=i+1)

    # Add bars for each segment in the "Par Value" row (row 2)
    for i, segment in enumerate(segments):
        fig.add_trace(go.Bar(name='Last Close', x=categories, y=par_last_close[segment]), row=2, col=i+1)
        fig.add_trace(go.Bar(name=f'{look_back}-Day Avg', x=categories, y=par_avg[segment]), row=2, col=i+1)
    
    # Add row titles as vertical annotations on the left
    fig.add_annotation(dict(x=-0.05, y=1, showarrow=False, text='Number of Trades', 
                            xref='paper', yref='paper', font=dict(size=12), textangle=-90, align='center'))
    fig.add_annotation(dict(x=-0.05, y=0.05, showarrow=False, text='Par Value (Billion)', 
                            xref='paper', yref='paper', font=dict(size=12), textangle=-90, align='center'))


    fig.update_layout(
        title=f'{title} - {last_date} vs {look_back}-Day Average',
        barmode='group',
        showlegend=True,
        height=500,  
        width=1300   
    )

    fig.show()



In [68]:
df = hist_trace(days=look_back)
df.head()

Data for 2024-09-02 is not available. Skipping this date.


Category ATS & Interdealer           Dealer to Customer  \
    Unnamed: 0_level_1            Trades Par Value             Trades   
0                 TIPS            1614.0       9.6             1911.0   
1           <= 5 years             889.0       6.3              874.0   
2           On-the-run             171.0       3.9              172.0   
3          Off-the-run             718.0       2.4              702.0   
4  > 5 and <= 10 years             413.0       2.7              667.0   

              Total                  Date  
  Par Value  Trades Par Value              
0      20.3  3525.0      29.9  2024-08-05  
1      12.6  1763.0      18.8  2024-08-05  
2       7.1   343.0      11.0  2024-08-05  
3       5.4  1420.0       7.8  2024-08-05  
4       6.6  1080.0       9.2  2024-08-05

In [75]:
df_TIPS = df[df[('Category', 'Unnamed: 0_level_1')] == 'TIPS'].reset_index(drop=True)
df_less_than_5 = get_subcat_rows(df, '<= 5 years')
df_5_10 = get_subcat_rows(df, '> 5 and <= 10 years')
df_greater_than_10 = get_subcat_rows(df, '> 10 years')

plot_trades_and_par(df_TIPS, 'TIPS')
plot_trades_and_par(df_less_than_5, 'Less Than 5 Years')
plot_trades_and_par(df_5_10, '5 to 10 Years')
plot_trades_and_par(df_greater_than_10, 'Greater Than 10 Years')